## Tracking

In [1]:
import cv2
import csv
import os
import numpy as np

# Global variables for custom ROI selection
circle_center = None
circle_radius = 0
drawing = False

def select_circle(event, x, y, flags, param):
    """Mouse callback function for selecting a circular ROI."""
    global circle_center, circle_radius, drawing

    if event == cv2.EVENT_LBUTTONDOWN:
        # Start drawing
        circle_center = (x, y)
        circle_radius = 0
        drawing = True

    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        # Update radius as the mouse moves
        circle_radius = int(np.sqrt((x - circle_center[0])**2 + (y - circle_center[1])**2))

    elif event == cv2.EVENT_LBUTTONUP:
        # Finish drawing
        drawing = False

def custom_roi_selection(frame, title="Select Object"):
    """Custom circular ROI selection."""
    global circle_center, circle_radius, drawing

    clone = frame.copy()
    cv2.namedWindow(title)
    cv2.setMouseCallback(title, select_circle)

    while True:
        temp_frame = clone.copy()
        if circle_center and circle_radius > 0:
            # Draw the circle as the user selects
            cv2.circle(temp_frame, circle_center, circle_radius, (0, 255, 0), 2)
        cv2.imshow(title, temp_frame)

        # Break on Enter key
        key = cv2.waitKey(1) & 0xFF
        if key == 13:  # Enter key
            break
        elif key == 27:  # Escape key
            circle_center, circle_radius = None, 0
            break

    cv2.destroyWindow(title)
    if circle_center and circle_radius > 0:
        # Convert circle to bounding box
        x = circle_center[0] - circle_radius
        y = circle_center[1] - circle_radius
        w = circle_radius * 2
        h = circle_radius * 2
        return (x, y, w, h)
    else:
        return None

def process_video(video_path, start_frame):
    """Process a single video and return tracking results, continuing frame count from start_frame."""
    video = cv2.VideoCapture(video_path)
    tracking_data = []

    if not video.isOpened():
        print(f"Error: Could not open video {video_path}.")
        return tracking_data, start_frame

    # Read the first frame
    success, frame = video.read()
    if not success:
        print(f"Error: Could not read video {video_path}.")
        return tracking_data, start_frame

    # Select ROIs manually using the custom selector
    print("Select Object 1:")
    bbox1 = custom_roi_selection(frame, "Select Object 1")
    if not bbox1:
        print("Selection canceled for Object 1.")
        return tracking_data, start_frame

    print("Select Object 2:")
    bbox2 = custom_roi_selection(frame, "Select Object 2")
    if not bbox2:
        print("Selection canceled for Object 2.")
        return tracking_data, start_frame

    # Initialize trackers for the selected objects
    tracker1 = cv2.TrackerCSRT_create()
    tracker2 = cv2.TrackerCSRT_create()
    tracker1.init(frame, bbox1)
    tracker2.init(frame, bbox2)

    frame_count = 0
    processed_frame_count = 0
    frame_step = 20

    while True:
        
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_count)
        success, frame = video.read()
        if not success:
            print(f"End of video {video_path}.")
            break
            
        processed_frame_count += 1

        # Update trackers
        success1, bbox1 = tracker1.update(frame)
        success2, bbox2 = tracker2.update(frame)

        # Initialize coordinates as None
        x1, y1, x2, y2 = None, None, None, None

        if success1:
            (x1, y1, w1, h1) = [int(v) for v in bbox1]
            cv2.rectangle(frame, (x1, y1), (x1 + w1, y1 + h1), (255, 0, 0), 2)
            cv2.putText(frame, "Object 1", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        if success2:
            (x2, y2, w2, h2) = [int(v) for v in bbox2]
            cv2.rectangle(frame, (x2, y2), (x2 + w2, y2 + h2), (0, 255, 0), 2)
            cv2.putText(frame, "Object 2", (x2, y2 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Store frame data
        tracking_data.append([
            frame_count,
            x1,
            y1,
            x2,
            y2
        ])

        # Display the frame
        cv2.imshow("Tracking", frame)

        frame_count += frame_step

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video.release()
    cv2.destroyAllWindows()

    return tracking_data, frame_count

## Directory processing

In [ ]:
def process_directory(directory_path, output_csv):
    """Process all videos in a directory and save to a single CSV file with continuous frame counting."""
    # List video files with case-insensitive extension matching
    video_files = [f for f in os.listdir(directory_path) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]

    # Sort files to ensure consistent ordering
    video_files.sort()

    if not video_files:
        print("No video files found in the directory.")
        return

    print(f"Found {len(video_files)} video(s) in the directory.")

    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)

    # Create CSV file and write header
    with open(output_csv, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(['Frame', 'Object1_X', 'Object1_Y', 'Object2_X', 'Object2_Y'])

        # Process all videos sequentially
        current_frame = 0
        for video_file in video_files:
            video_path = os.path.join(directory_path, video_file)
            print(f"Processing {video_path}...")
            video_tracking_data, current_frame = process_video(video_path, current_frame)

            # Write data for this video
            writer.writerows(video_tracking_data)

    print(f"All tracking results saved to {output_csv}")

# Main directory processing
input_directory = R"C:\Users\barei\Documents\GitHub\Ants\input videos"
output_csv = R"C:\Users\barei\Documents\GitHub\Ants\output_csv\continuous_tracking_data_2024_09_19_S616.csv"
process_directory(input_directory, output_csv)

## Single file processing

In [4]:
def process_single_file(input_file_path, output_csv):
    """Process a single video file and save to a CSV file."""
    # Verify file extension
    if not input_file_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        raise ValueError("Unsupported file format. Please provide a video file (mp4, avi, mov, or mkv)")

    # Verify file exists
    if not os.path.exists(input_file_path):
        raise FileNotFoundError(f"Video file not found: {input_file_path}")

    print(f"Processing file: {input_file_path}")

    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)
    
    current_frame = 0

    # Create CSV file and write header
    with open(output_csv, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(['Frame', 'Object1_X', 'Object1_Y', 'Object2_X', 'Object2_Y'])

        # Process the video
        video_tracking_data, current_frame = process_video(input_file_path, current_frame)

        # Write data for this video
        writer.writerows(video_tracking_data)

    print(f"Tracking results saved to {output_csv}")

# Example usage:
input_file = R"C:\Users\barei\Documents\GitHub\Ants\input videos\S6160003.MP4"
output_csv = R"C:\Users\barei\Documents\GitHub\Ants\Analyzed Data\2024_09_23\s6160003\coordinates.csv"
process_single_file(input_file, output_csv)

Processing file: C:\Users\barei\Documents\GitHub\Ants\input videos\S6160003.MP4
Select Object 1:
Select Object 2:
End of video C:\Users\barei\Documents\GitHub\Ants\input videos\S6160003.MP4.
Tracking results saved to C:\Users\barei\Documents\GitHub\Ants\Analyzed Data\2024_09_23\s6160003\coordinates.csv
